# Modelo de predicción de Categoría

Este modelo es entrenado usando los datos de tweets proveídos por Hey Banco, sobre los tweets que reciben en su cuenta de twitter, los cuales varían entre solicitudes de ayuda, hasta comentarios negativos.

Este modelo busca categorizar futuros tweets en alguna de las siguientes categorías:

- Agradecimiento
- Solicitud de Apoyo
- Comentario Positivo
- Comentario Negativo
- Caso Resuelto
- Comentario Neutral
- Promociones

Si el comentario no cae dentro de alguno de estas categorías, se le aplicará la categoría de "Otros".

**Este modelo categoriza correctamente alrededor del 67% de los tweets**

In [ ]:
%pip install numpy scipy nlpaug
%pip install -r requirements.txt

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
# Cargar los datos de las columnas de "tweet" y "Categoria 1" para entrenar el modelo
data = pd.read_csv('data.csv', usecols=["tweet", "Categoria 1"])

# Eliminamos los registros sin tweets o sin categorías
data.dropna(subset=['tweet'], inplace=True)
data = data[(data['Categoria 1'] != '?') & (data['Categoria 1'].notna()) & (data['Categoria 1'] != 'Otros')]

data.head()

In [ ]:
# Partir los datos para el entrenamiento y para probar (80% para entrenamiento, 20% para probar)
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['Categoria 1'], test_size=0.2, random_state=42)

# Vectorizar los tweets usando TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Entrenamiento del modelo SVC
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_vectorized, y_train)

# Predecir las categorías para los datos de prueba
predictions = svm_classifier.predict(X_test_vectorized)

# Evaluar la eficacia del modelo
accuracy = accuracy_score(y_test, predictions)
print('Accuracy:', accuracy)

Ya que se tiene el modelo entrenado, se puede cargar un archivo .csv que contenga la columna "tweet".

El código leerá el archivo y usando el modelo creado anteriormente, agregará una columna nueva llamada "predicted_category" con las predicciones de la categoría.

In [ ]:
# Cargar el archivo a una DataFrame
input = pd.read_csv('tweets.csv')

# Vectorizar los tweets nuevos
X_data_vectorized = vectorizer.transform(input['tweet'])

# Predecir las categorias de los tweets
predicted_categories = svm_classifier.predict(X_data_vectorized)

# Agregar las predicciones al DataFrame
input['predicted_category'] = predicted_categories

# Exportar el DataFrame a un archivo .csv
input.to_csv('tweets_with_predictions.csv', index=False)
